In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')
import numpy as np
# Para la visualización 
# -----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
# Otros objetivos
# -----------------------------------------------------------------------
import math
from itertools import combinations
# Para pruebas estadísticas
# -----------------------------------------------------------------------
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
# Para la codificación de las variables numéricas
# -----------------------------------------------------------------------
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder # para poder aplicar los métodos de OneHot, Ordinal,  Label y Target Encoder 
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder, CountEncoder
import sys
sys.path.append("../../")
import src.soporte_encoding  as se
from src.soporte_graficas import (
    visualizar_categoricas
)
from src.SupportPreProcesamiento import (
    separarar_df
)

In [2]:
df = pd.read_csv("../../datos/03_api_rent_sin_nulos.csv")

# Encoding
- Vamos a pasar todas las variables a numéricas
- Hay que saber si tienen orden o no

### Pasar las columnas a su tipo correcto

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   propertyType      353 non-null    object 
 1   price             353 non-null    float64
 2   size              353 non-null    float64
 3   rooms             353 non-null    object 
 4   bathrooms         353 non-null    object 
 5   floor             353 non-null    object 
 6   municipality      353 non-null    object 
 7   hasLift           353 non-null    object 
 8   distancia_centro  353 non-null    object 
dtypes: float64(2), object(7)
memory usage: 24.9+ KB


In [4]:
df["rooms"] = df["rooms"].astype("O")
df["bathrooms"] = df["bathrooms"].astype("O")

# Analicemos estadísticamente
- Dos opciones: mannwhitneyu
- Mas de dos opciones: Kruskal

No es una distribución normal

In [5]:
numericas,categoricas = separarar_df(df)

In [6]:
# instanciamos la clase de Asunciones, para poder decir qué método usar para el análisis estadístico
asunciones = se.Asunciones(dataframe = df, columna_numerica = "price")
print("\n-------------\n")


for categoria in categoricas:   

    print(f"Estamos analizando la variable {categoria.upper()}")
    
    asunciones.identificar_normalidad(metodo = "kolmogorov") 

    # comprobamos la homogeneidad de varianzas
    asunciones.identificar_homogeneidad(columna_categorica = categoria)

    # instanciamos la clase para evaluar si hay diferencias entre los distintos grupos de las variables categóricas
    test = se.TestEstadisticos(df, "price", categoria)
    test.run_all_tests()
    print("\n###########################\n")


-------------

Estamos analizando la variable PROPERTYTYPE
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable propertyType las varianzas NO son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['flat', 'penthouse', 'studio', 'duplex', 'chalet', 'countryHouse']

Realizando test de Kruskal...
Estadístico de prueba: 10.491276765664804
Valor p: 0.062453362538402654
No hay evidencia suficiente para concluir que hay una diferencia significativa.
Los resultados del test de Tukey son: 



,meandiff,p-adj,lower,upper,group_diff
0,-10.0000,1.0000,-207.3147,187.3147,chalet-countryHouse
1,79.2857,0.2949,-29.4294,188.0008,chalet-duplex
2,87.5596,0.1184,-11.6306,186.7497,chalet-flat
3,9.9000,0.9999,-102.5867,122.3867,chalet-penthouse
4,79.6250,0.2206,-22.0687,181.3187,chalet-studio
5,89.2857,0.6984,-87.5915,266.1629,countryHouse-duplex
6,97.5596,0.5771,-73.6282,268.7473,countryHouse-flat
7,19.9000,0.9996,-159.3200,199.1200,countryHouse-penthouse
8,89.6250,0.6724,-83.0254,262.2754,countryHouse-studio
9,8.2739,0.9959,-38.5355,55.0832,duplex-flat



###########################

Estamos analizando la variable ROOMS
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable rooms las varianzas son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['1 habitacion', '2 habitaciones', '3 habitaciones', 'sin habitaciones', '4 habitaciones']

Realizando test de Kruskal...
Estadístico de prueba: 6.997445029790361
Valor p: 0.13602330549397834
No hay evidencia suficiente para concluir que hay una diferencia significativa.
Los resultados del test de Tukey son: 



,meandiff,p-adj,lower,upper,group_diff
0,15.3159,0.2789,-5.9159,36.5476,1 habitacion-2 habitaciones
1,22.5779,0.1712,-5.1881,50.3438,1 habitacion-3 habitaciones
2,30.6804,0.9113,-67.1878,128.5487,1 habitacion-4 habitaciones
3,8.2043,0.9324,-20.1500,36.5585,1 habitacion-sin habitaciones
4,7.2620,0.9511,-20.2748,34.7988,2 habitaciones-3 habitaciones
5,15.3646,0.9928,-82.4389,113.1680,2 habitaciones-4 habitaciones
6,-7.1116,0.9579,-35.2415,21.0183,2 habitaciones-sin habitaciones
7,8.1026,0.9994,-91.3242,107.5293,3 habitaciones-4 habitaciones
8,-14.3736,0.7616,-47.7121,18.9648,3 habitaciones-sin habitaciones
9,-22.4762,0.9720,-122.0688,77.1164,4 habitaciones-sin habitaciones



###########################

Estamos analizando la variable BATHROOMS
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable bathrooms las varianzas NO son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['1 aseo', '2 aseos', '3 aseos']

Realizando test de Kruskal...
Estadístico de prueba: 2.17914787981004
Valor p: 0.3363597726585303
No hay evidencia suficiente para concluir que hay una diferencia significativa.
Los resultados del test de Tukey son: 



...


###########################

Estamos analizando la variable HASLIFT
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable hasLift las varianzas son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['tiene ascensor', 'no tiene ascensor', 'desconocido']

Realizando test de Kruskal...
Estadístico de prueba: 13.627872283280148
Valor p: 0.0010983610754482008
Hay una diferencia significativa entre los grupos
Los resultados del test de Tukey son: 



,meandiff,p-adj,lower,upper,group_diff
0,37.4113,0.0345,2.1662,72.6565,desconocido-no tiene ascensor
1,52.0307,0.0016,16.9181,87.1432,desconocido-tiene ascensor
2,14.6193,0.0694,-0.8887,30.1274,no tiene ascensor-tiene ascensor



###########################

Estamos analizando la variable DISTANCIA_CENTRO
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable distancia_centro las varianzas NO son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['Entre 5 y 10 km', 'Entre 10 y 20 km', 'Mas de 50 km', 'Entre 30 y 40 km', 'Entre 40 y 50 km', 'Entre 1 y 5 km', 'Entre 20 y 30 km', 'Menos de 1 km']

Realizando test de Kruskal...
Estadístico de prueba: 50.242312632000186
Valor p: 1.2945889652583908e-08
Hay una diferencia significativa entre los grupos
Los resultados del test de Tukey son: 



,meandiff,p-adj,lower,upper,group_diff
0,1.8831,1.0000,-27.1857,30.9520,Entre 1 y 5 km-Entre 10 y 20 km
1,-1.7755,1.0000,-29.9682,26.4172,Entre 1 y 5 km-Entre 20 y 30 km
2,-31.7823,0.2155,-71.1487,7.5842,Entre 1 y 5 km-Entre 30 y 40 km
3,-71.3656,0.0000,-107.7836,-34.9476,Entre 1 y 5 km-Entre 40 y 50 km
4,10.2604,0.9486,-17.2982,37.8190,Entre 1 y 5 km-Entre 5 y 10 km
5,-204.0323,0.0000,-288.5065,-119.5580,Entre 1 y 5 km-Mas de 50 km
6,-12.3656,0.9960,-64.0081,39.2770,Entre 1 y 5 km-Menos de 1 km
7,-3.6586,0.9999,-31.4950,24.1778,Entre 10 y 20 km-Entre 20 y 30 km
8,-33.6654,0.1505,-72.7775,5.4467,Entre 10 y 20 km-Entre 30 y 40 km
9,-73.2487,0.0000,-109.3916,-37.1058,Entre 10 y 20 km-Entre 40 y 50 km



###########################



# Resultados
### Variables ordinales
- floor
- municipality
- hasLift
### Variables Nominales
- propertyType
- rooms
- bathrooms

# Tenemos Ordinales y Nominales
- Las ordinales las gestionaremos con Target Encoding, que codifica con la media del objetivo para la categoría
- Para las nominales usaremos one-Hot Encoding

In [7]:
df.sample()

,propertyType,price,size,rooms,bathrooms,floor,municipality,hasLift,distancia_centro
262,countryHouse,600.0,60.0,2 habitaciones,2 aseos,desconocido,Clase Alta,desconocido,Entre 20 y 30 km


# Ordinales

In [8]:
cols_ordinales = ["floor","municipality","hasLift","distancia_centro"]

### Target Encoder

In [9]:
target_encoder = TargetEncoder(cols=cols_ordinales)
df = target_encoder.fit_transform(df,df[["price"]])

In [10]:
df.sample()

,propertyType,price,size,rooms,bathrooms,floor,municipality,hasLift,distancia_centro
340,flat,720.0,58.0,2 habitaciones,1 aseo,691.264721,677.821984,687.633544,703.300204


# Nominales

In [11]:
cols_nominales = ["propertyType","rooms","bathrooms"]

### OneHotEncoder

In [12]:
encoder = OneHotEncoder(categories='auto', 
                        drop=None, 
                        sparse_output=True, 
                        dtype='float', 
                        handle_unknown='error')

# Ajustar el codificador a los datos y transformarlos
encoder_trans = encoder.fit_transform(df[cols_nominales])

# lo siguiente que hacemos es convertir el objeto devuelto por el fit_transform a array para poder verlo
encoder_array = encoder_trans.toarray()

# usaremos el método '.get_feature_names_out()' para extraer el nombre de las columnas
nombre_columnas = encoder.get_feature_names_out()

# creamos un DataFrame con los resultados obtenidos de la transformación
encoder_df = pd.DataFrame(encoder_array, columns = nombre_columnas)

# concatenamos estos resultados con el DataFrame original
df = pd.concat([df, encoder_df], axis = 1)

In [13]:
df.sample()

,propertyType,price,size,rooms,bathrooms,floor,municipality,hasLift,distancia_centro,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio,rooms_1 habitacion,rooms_2 habitaciones,rooms_3 habitaciones,rooms_4 habitaciones,rooms_sin habitaciones,bathrooms_1 aseo,bathrooms_2 aseos,bathrooms_3 aseos
151,flat,750.0,54.0,1 habitacion,1 aseo,703.740243,701.840488,702.252872,703.300204,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# Eliminar columnas nominales
- Ya tenemos las columnas generadas y ya no nos aportan información

In [14]:
df.drop(columns=cols_nominales,inplace=True)

In [15]:
df.sample()

,price,size,floor,municipality,hasLift,distancia_centro,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio,rooms_1 habitacion,rooms_2 habitaciones,rooms_3 habitaciones,rooms_4 habitaciones,rooms_sin habitaciones,bathrooms_1 aseo,bathrooms_2 aseos,bathrooms_3 aseos
124,600.0,90.0,670.416613,677.821984,702.252872,699.726071,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


# Guardamos
- Nos vamos a feature scaling con este df

In [16]:
df.to_csv("../../datos/04_rent_target_onehot_encoded.csv",index=False)